### Import Dependencies

In [1]:
import openai
import instructor
from qdrant_client import QdrantClient

from pydantic import BaseModel, Field

### RAG Pipeline

In [2]:
client = instructor.from_openai(openai.OpenAI())

In [3]:
class RAGGenerationRepsonse(BaseModel):
    answer: str = Field(description="The answer to the question")

In [4]:
def get_embeddings(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model
    )

    return response.data[0].embedding

def retrieve_data(query, qdrant_client, k=5):
    query_embedding = get_embeddings(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k
    )

    retrieved_context_ids = []
    retrieved_context= []
    similiarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload['parent_asin'])
        retrieved_context.append(result.payload['description'])
        similiarity_scores.append(result.score)
        retrieved_context_ratings.append(result.payload['average_rating'])

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similiarity_scores": similiarity_scores,
        "retrieved_context_ratings": retrieved_context_ratings
    }

def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context['retrieved_context_ids'], context['retrieved_context'], context['retrieved_context_ratings']):
        formatted_context += f"- ID: {id}, Rating: {rating}, Description: {chunk}\n"

    return formatted_context

def build_prompt(preprocessed_context, question):
    prompt = f"""
    You are a shopping assistent that can answer questions about the products in stock.
    
    You will be given a question and a list of context.
    
    Instructions:
    - You need to answer the questions based on the provided context only.
    - Never use word context and refer to it as a the available products.
    
    Context:
    {preprocessed_context}
    
    Question:
    {question}
    """

    return prompt

def generate_answer(prompt):

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        response_model=RAGGenerationRepsonse
    )

    return response

def rag_pipeline(question, qdrant_client, k=5):

    retrieved_context = retrieve_data(question, qdrant_client, k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "datamodel": answer,
        "answer": answer.answer,
        'question': question,
        "retrieved_context_ids": retrieved_context['retrieved_context_ids'],
        "retrieved_context": retrieved_context['retrieved_context'],
        "similiarity_scores": retrieved_context['similiarity_scores']
    }

    return final_result

In [5]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [6]:
output = rag_pipeline("Can I get charging cable, please suggest me a few good ones", qdrant_client)

In [7]:
output

{'datamodel': RAGGenerationRepsonse(answer='Here are a few good charging cables available:\n\n1. iPhone Charger Lightning Cable 3Pack Quick Charger Rapid Cord Apple MFi Certified - Compatible with iPhone 11 Pro, X, XR, XS MAX, 8 Plus, 7, 6s, 5s, 5c, Air, Mini, iPod. (Rating: 4.3)\n\n2. BLACKSYNCZE USB C to Lightning Cable 2Pack 6FT, MFi Certified - Fast charging cable with 20W PD fast charging, durable nylon braided design, compatible with iPhone 13, 12, 11 Pro Max, XR, XS, X, 8, 7, 6s, SE2020, and more. (Rating: 4.4)\n\nBoth options are highly rated and suitable for various iPhone models.'),
 'answer': 'Here are a few good charging cables available:\n\n1. iPhone Charger Lightning Cable 3Pack Quick Charger Rapid Cord Apple MFi Certified - Compatible with iPhone 11 Pro, X, XR, XS MAX, 8 Plus, 7, 6s, 5s, 5c, Air, Mini, iPod. (Rating: 4.3)\n\n2. BLACKSYNCZE USB C to Lightning Cable 2Pack 6FT, MFi Certified - Fast charging cable with 20W PD fast charging, durable nylon braided design, comp

In [8]:
print(output['answer'])

Here are a few good charging cables available:

1. iPhone Charger Lightning Cable 3Pack Quick Charger Rapid Cord Apple MFi Certified - Compatible with iPhone 11 Pro, X, XR, XS MAX, 8 Plus, 7, 6s, 5s, 5c, Air, Mini, iPod. (Rating: 4.3)

2. BLACKSYNCZE USB C to Lightning Cable 2Pack 6FT, MFi Certified - Fast charging cable with 20W PD fast charging, durable nylon braided design, compatible with iPhone 13, 12, 11 Pro Max, XR, XS, X, 8, 7, 6s, SE2020, and more. (Rating: 4.4)

Both options are highly rated and suitable for various iPhone models.


### RAG Pipeline With Grounding Context

In [9]:
class RAGUsedContext(BaseModel):
    id: str = Field(description="The id of the item used to answer the question")
    description: str = Field(description="Short description of the item used to answer the question")

class RAGGenerationRepsonse(BaseModel):
    answer: str = Field(description="The answer to the question")
    reference: list[RAGUsedContext] = Field(description="The list of items used to answer the question")  

In [10]:
def get_embeddings(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model
    )

    return response.data[0].embedding

def retrieve_data(query, qdrant_client, k=5):
    query_embedding = get_embeddings(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k
    )

    retrieved_context_ids = []
    retrieved_context= []
    similiarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload['parent_asin'])
        retrieved_context.append(result.payload['description'])
        similiarity_scores.append(result.score)
        retrieved_context_ratings.append(result.payload['average_rating'])

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similiarity_scores": similiarity_scores,
        "retrieved_context_ratings": retrieved_context_ratings
    }

def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context['retrieved_context_ids'], context['retrieved_context'], context['retrieved_context_ratings']):
        formatted_context += f"- ID: {id}, Rating: {rating}, Description: {chunk}\n"

    return formatted_context

def build_prompt(preprocessed_context, question):
    prompt = f"""
    You are a shopping assistent that can answer questions about the products in stock.
    
    You will be given a question and a list of context.
    
    Instructions:
    - You need to answer the questions based on the provided context only.
    - Never use word context and refer to it as a the available products.
    - As an output, you need to provide:

    * The answer to the question based on provided context
    * The list of the IDs of the chunks that were used to answer the question. Only return the ones that are used in the answer.
    * Short description (1-2 sentences) of the chunks that were used to answer the question.

    - The short description should have the name of the item.
    - The answer to the question should contain detailed information about the product and returned with detailed specification in bullet points.
    
    Context:
    {preprocessed_context}
    
    Question:
    {question}
    """

    return prompt

def generate_answer(prompt):

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        response_model=RAGGenerationRepsonse
    )

    return response

def rag_pipeline(question, qdrant_client, k=5):

    retrieved_context = retrieve_data(question, qdrant_client, k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "original_output": answer,
        "answer": answer.answer,
        "reference": answer.reference,
        'question': question,
        "retrieved_context_ids": retrieved_context['retrieved_context_ids'],
        "retrieved_context": retrieved_context['retrieved_context'],
        "similiarity_scores": retrieved_context['similiarity_scores']
    }

    return final_result

In [11]:
result = rag_pipeline("Can I get some earphones", qdrant_client, k=10)

In [12]:
result

{'original_output': RAGGenerationRepsonse(answer='Yes, you can get several types of earphones from the available products:\n\n1. Tronsmart Wireless Earbuds (ID: B0B3HYZ427):\n- Dual-driver technology with 10mm Dynamic Driver and balanced Armature Driver for high and low-frequency sound.\n- Bluetooth 5.3 with 3 EQ modes (Bass Booster, Pure, Normal).\n- IPX5 waterproof rating.\n- 32 hours total playtime (7 hours per charge plus charging case).\n- Ergonomic, lightweight design with 3 sizes of ear tips.\n- Compatible with most Bluetooth devices.\n\n2. Bluetooth Headphones Wireless Earbuds (ID: B0C3ZQ7KHJ):\n- 90 hours total playtime with 2200mAh charging case.\n- Dual LED digital power display.\n- 11mm speaker with dual-chamber acoustic drivers.\n- IPX7 sweatproof.\n- Flexible earhooks and multiple ear tip sizes for secure fit.\n- Supports wireless and Type-C charging.\n\n3. VOFUOE Bone Conduction Bluetooth Headphones (ID: B0B2W7QJXT):\n- Bone conduction technology transmitting sound throu

In [13]:
print(result['answer'])

Yes, you can get several types of earphones from the available products:

1. Tronsmart Wireless Earbuds (ID: B0B3HYZ427):
- Dual-driver technology with 10mm Dynamic Driver and balanced Armature Driver for high and low-frequency sound.
- Bluetooth 5.3 with 3 EQ modes (Bass Booster, Pure, Normal).
- IPX5 waterproof rating.
- 32 hours total playtime (7 hours per charge plus charging case).
- Ergonomic, lightweight design with 3 sizes of ear tips.
- Compatible with most Bluetooth devices.

2. Bluetooth Headphones Wireless Earbuds (ID: B0C3ZQ7KHJ):
- 90 hours total playtime with 2200mAh charging case.
- Dual LED digital power display.
- 11mm speaker with dual-chamber acoustic drivers.
- IPX7 sweatproof.
- Flexible earhooks and multiple ear tip sizes for secure fit.
- Supports wireless and Type-C charging.

3. VOFUOE Bone Conduction Bluetooth Headphones (ID: B0B2W7QJXT):
- Bone conduction technology transmitting sound through cheekbones.
- Open-ear design for comfort and safety.
- Bluetooth 